In [1]:
import pandas as pd
import numpy as np
import sklearn 
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from random import shuffle
from scipy.stats import ranksums
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

In [2]:
# Defino variables donde voy a ir a poner cosas 
acc = []
acc_sh = []
cm = np.zeros((2,2))
cm_sh = np.zeros((2,2))
features_selected = np.zeros(100)
features_selected_sh = np.zeros(100)

numfeat = 50 # numero de features para usar con feature selection univariado 


In [3]:

# Este codigo existe para seleccionar un threshold que haga a las matrices de confusion lo mas diagonales posibles
def diag_cm(classifier, X_test_new): 
    thr = np.arange(0,1,0.1)
    opt = 10*len(X_test_new)
    for t in thr:
        pred = classifier.predict_proba(X_test_new)[:,1]>t
        cm_temp = np.double(np.array(confusion_matrix(y[test],pred)))
        opt_temp = np.abs(cm_temp[0,0] - cm_temp[1,1])+cm_temp[0,1]+cm_temp[1,0]
        if opt_temp<opt:
            cm_out = cm_temp
            opt=opt_temp
           # print(opt)
           # print(cm_out)
    return cm_out




In [38]:
# 1. cargo la matriz con features

#DATA CN DEBERIA SER DATOS DE PERSONAS SANAS Y CON LOS MISMOS FEATURES QUE LAS PERSONAS ENFERMAS, ICTAL
data_CN = np.array(pd.read_csv('D:/Alessandro/LACONEU/InterIctalA.csv', delimiter=',',header=0))
data_AD = np.array(pd.read_csv('D:/Alessandro/LACONEU/Ictal2.csv',delimiter=',',header=0))

data_CN = np.delete(data_CN, 0, 1)
data_AD = np.delete(data_AD, 0, 1)

#print(data_CN)
#print(data_AD)

In [40]:
# 2. Crear matriz con clases
y_CN = np.zeros(data_CN.shape[0])
y_AD = np.ones(data_AD.shape[0])

y = np.concatenate((y_CN,y_AD))
X = np.concatenate((data_CN,data_AD))

# X datos matriz de features
# y vector de etiquetas

print("MATRIZ DE FEATURES")
print(X)

print("VECTOR DE ETIQUETAS")
print(y)

MATRIZ DE FEATURES
[[0.82872823 0.55658427 0.51813273 ... 2.40871208 1.29684207 1.39661611]
 [0.82019924 0.63590208 0.58239609 ... 2.32371155 1.30156915 1.18732333]
 [0.78588985 0.59862576 0.55779447 ... 2.26031552 1.24199278 1.2121708 ]
 ...
 [0.64959457 0.47716719 0.4458073  ... 2.43123945 1.20734358 1.46841019]
 [0.73646508 0.35493727 0.39069644 ... 2.2627397  1.17585623 1.64797913]
 [0.76228874 0.50865544 0.49326597 ... 1.94938296 1.26711314 1.42967687]]
VECTOR DE ETIQUETAS
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]


## Generar Esquema de particion - Cross Validation